# The Gerchberg Saxton Algorithm <a id="top"></a>

- The Gerchberg Saxton (GS) Algorithm (1972) is a reconstructive technique based on the idea of alternating projections between the image and diffraction planes using the Fourier Transform (FT). In this file, we will code an example of the Gerchberg Saxton Algorithm and attempt to implement some techniques such as zero packing and a modification of zero packing.

In [1]:
import numpy as np
from scipy import linalg as la
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import copy
import time 
from datetime import date

In [2]:
today = date.today()

**Table of Contents:**
   * [Zero Packing Function](#zp-imp)
   * ["Unpacking" Function](#unpack)
   * [Initial Conditions to Problem](#ic)
   
   
   * [Original Function](#o-fun)
       * [Random Initial Phase](#ran-init)
       * [Magnitudes](#mag)
       * [Phases](#phases)
       * [Gerchberg Saxton Algorithm](#gsa)
       * [Estimated Phases & Error](#e-phases-error)
       * [Convergence of Error](#conv-error)
       

   * [Zero Packed Function](#zp-fun)
       * [Random Initial Phase](#ran-init-zp)
       * [Magnitudes](#mag-zp)
       * [Phases](#phases-zp)
       * [Gerchberg Saxton Algorithm](#gsa)
       * [Estimated Phases & Error](#e-phases-error-zp)
       * [Convergence of Error](#conv-error-zp)


   * [Plots](#plots)
       * [Original Function Plots](#o-plots)
       * [Zero Packed Function Plots](#plots-zp)

-------------------

## Zero Packing Function: <a id="zp"></a>

In [3]:
# z represents how many zeros you can to put after each number
# thing represents the list or array you are trying to zero pack
def zero_packing(thing, z): 
    if type(thing) == np.ndarray:
        thing = thing.tolist()
        #print(thing)
    # How many elements in the list before zero packing
    #ele = len(thing)
    # creates a deep copy so it wont effect the original list
    zero_pack = copy.deepcopy(thing)
    
    n = 2
    while n <= z+1:
        for i in range(len(thing)):
            zero_pack.insert(n*i+1, 0)
        n += 1
    zero_pack = np.array(zero_pack)
    return zero_pack

## "Unpacking" Function <a id="unpack"></a>

In [4]:
# Thing refers to the zero-packed list or array that will now be unpacked
def unpacking(zp_thing, z):
    unpack = copy.deepcopy(zp_thing)
    unpack = np.array(unpack[::z+1])
    return unpack

## Modified Zero Packing 
* This method will take the original vector f and replace every zth term with a zero

In [5]:
# this method packs every zth element with a zero and also tries to follow
# the zero packing pattern
def zero_packing1(thing, z): 
    if type(thing) == np.ndarray:
        thing = thing.tolist()

    # Uses the original list and takes each z+1 component
    zero_pack_small = thing[::z+1]
    
    # records the number of elements from the new list
    num_ele = len(zero_pack_small)

    n = 2
    while n <= z+1:
        for i in range(num_ele-1):
            zero_pack_small.insert(n*i+1, 0)
        n += 1
        
    # if the length of the original list and the new list do not match up: make them the same length.
    if len(thing) != len(zero_pack_small):
        add = abs(len(thing)-len(zero_pack_small))
        for i in range(add):
            zero_pack_small.append(0)
    
    zero_pack = np.array(zero_pack_small)
    
    return zero_pack

--------------------

## Initial Conditions: <a id="ic"></a>

In [6]:
# z equals the number of zeros to pack
z = 1

#--- Given from Problem ---

# Sampling Rate:
delta = 1 / 128

# number of sampling points:
n = 256

# maximum number of iterations:
maxiter = 256
#maxiter = 500
t = np.linspace(-1, 1, num=n)
t_zp = np.linspace(-1, 1, num=z*n)

In [7]:
# rectangle function
def rect(t):
    y = np.zeros(len(t), int)
    for i in range(len(t)):
        if abs(t[i]) < 0.5:
            y[i] = 1
        else:
            y[i] = 0
    return y

-------------------

# Original Function<a id="o-fun"></a>

In [8]:
# function f
f = rect(2 * t) * np.exp(30 * np.pi * 1j * t ** 2)

#debugging #1: everything that is equal to -0 will now be zero
ffix1 = np.where(f == -0)
f[ffix1] = 0

#debugging #2: Sets all the values that are supposed to be zero to zero
ffix2 = np.where(f == np.finfo(float).eps)
f[ffix2] = 0

# Zero Packed Function<a id="zp-fun"></a>

In [9]:
f_zp = zero_packing(f, z)

# Modified Zero Packed Function

In [45]:
# zp modification 1
f_zp1 = zero_packing1(f,z)

-------

## Original Phases <a id="phases"></a>

In [11]:
# phases of f: angles (of our original phases)
# check if Arg(f)
phases = np.zeros(len(f))
for i in range(len(f)):
    phases[i] = np.angle(f[i])
    if phases[i] < 0:
        phases[i] += 2 * np.pi

## Zero Packed Phases <a id="phases-zp"></a>

In [12]:
phases_zp = np.zeros(len(f_zp))
for i in range(len(f_zp)):
    phases_zp[i] = np.angle(f_zp[i])
    if phases_zp[i] < 0:
        phases_zp[i] += 2 *np.pi

## Modified Zero Packed Phases

In [13]:
phases_zp1 = np.zeros(len(f_zp1))
for i in range(len(f_zp1)):
    phases_zp1[i] = np.angle(f_zp1[i])
    if phases_zp1[i] < 0:
        phases_zp1[i] += 2 *np.pi       

-------------------

## Magnitudes <a id="mag"></a>

In [14]:
# |f| magnitude in image plane
# recall, magnitudes of vectors is the "length" of a vector, aslo denoted b ||f||
f_mag = abs(f)

# |F| magnitude in diffraction plane
F_mag = abs(np.fft.fft(f)/len(f))

## Zero-Packed Magnitudes <a id="mag-zp"></a>

In [15]:
# |f| magnitude in image plane
f_mag_zp = abs(f_zp)

# |F| magnitude in diffraction plane
#F_mag_zp = abs(np.fft.fft(f_zp)/len(f_zp))
F_mag_zp = abs(np.fft.fft(f_zp)/len(f))

#for i in range(0, ):
#    F_mag_zp

## Modified Zero Packed Magnitudes

In [16]:
f_mag_zp1 = abs(f_zp1)

#F_mag_zp1 = abs(np.fft.fft(f_zp1)/len(f_zp1))
F_mag_zp1 = zero_packing1(F_mag, z)

## Modification #2 Zero Packed Magnitudes

In [17]:
# This is where the diffraction plane magnitudes are the same as the original 
# diffraction plane magnitudes, but every zth element is replaced with zero
F_mag_zp1_edit = abs(np.fft.fft(f_zp1)/len(f_zp1))

---------

## Random Initial Phase Generator:<a id="rand-init"></a>

In [18]:
# random phases to start algorithm
# uniform random distribution
omg = 2 * (np.pi * np.random.random(size=(len(f)))) - np.pi
#omg = unpacking(omg_zp, z)


In [19]:
# Normal Phase Generator
omg_normal = np.array([np.pi] * len(f))

### Zero Packing: Random Initial Phase Generator <a id="rand-init-zp"></a>

In [46]:
omg_zp = zero_packing(omg, z)

In [47]:
#normal phase generator:
omg_zp_normal = zero_packing(omg_normal, z)

### Modified Zero Packing: Random Initial Phase Generator:

In [48]:
# zp modification 1:
omg_zp1 = zero_packing1(omg, z)

In [49]:
#normal phase generator:
omg_zp1_normal = zero_packing1(omg_normal, z)

-------------------

# The Gerchberg Saxton Algorithm: <a id="gsa"></a>

In [50]:
N = len(f_mag)
N_zp = len(f_mag)
N_zp1 = len(f_mag_zp1)

In [51]:
# Takes in the maximum number of iterations, random iniial phases, mag of f and F.
def GSA(omg, f_mag, F_mag):
    N = len(f_mag)
    # Error Tolerance
    err_tol = 10 ** -10
    
    # where the error for each iteration will be stored
    # The GS Algorithm computes the error twice per iteration
    ##err = np.zeros(2 * maxiter + 1)
    #err =  [0,1]
    err = []
    # First estimate in image plane
    x = f_mag * np.exp(1j * omg)
    k = 0

    start = time.time()
    
    ##################################
    X = np.fft.fft(x) / N
    err.append(np.sqrt(N) * la.norm((abs(X) - F_mag), 2))
    Y = F_mag * np.exp(1j * np.angle(X))
    y = N * np.fft.ifft(Y)
    err.append(la.norm((abs(y) - f_mag), 2))
    x = f_mag * np.exp(1j * np.angle(y))
    k += 1
    ##################################
    
    while (abs(err[k-1] - err[k]) > err_tol and abs(err[k]) > err_tol) and k <= maxiter:
    
    #while k == 1 or ((abs(err[k-1] - err[k]) > err_tol or abs(err[k]) > err_tol) and k <= maxiter):
        
        # Step 1: Equation 5 (Estimate after Fourier Transform)
        # X is the estimate in the diffraction plane
        X = np.fft.fft(x) / N

        # Error is defined as the diff between estimated magnitude (X) and known magnitude (F_mag)
        # Diffraction plane error
        
        #err[k] = np.sqrt(N) * la.norm((abs(X) - F_mag), 2)
        err.append(np.sqrt(N) * la.norm((abs(X) - F_mag), 2))
        # Step 2: (Equation 6)
        Y = F_mag * np.exp(1j * np.angle(X))

        # Step 3: (Equation 7) IDTF of Y_k
        y = N * np.fft.ifft(Y)

        # Image plane (object domain) error
        #err[k + 1] = la.norm((abs(y) - f_mag), 2)
        err.append(la.norm((abs(y) - f_mag), 2))
        # Step 4: (Equation 4) Change first estimate to then begin iterative process:
        x = f_mag * np.exp(1j * np.angle(y))

        # increment index
        k += 1
        
        #Just making sure algorithm is working correctly
        #if ((abs(err[k-1] - err[k]) <= err_tol) and (abs(err[k]) <= err_tol)) or k > maxiter:
        #    print("watch out!")
        
    print(k)    
    ### These formulas are true because of Parseval's Theorem/Identity, which states that FT is a unitary operator ####
    
    # Estimated phases:
    e_phases = np.angle(x)
    
    # Doing necessary changes to recovered phases:
    # manually corrects estimated phases to 0 if they are equal to pi

    fix1 = np.where(e_phases == np.pi)
    fix2 = np.where(e_phases == -np.pi)
    #fix3 = np.where(e_phases < 0)
    #fix4 = np.where(e_phases == -0)
    #fix5 = np.where(e_phases > np.pi)
    
    e_phases[fix1] = 0
    e_phases[fix2] = 0
    #e_phases[fix3] += 2*np.pi
    #e_phases[fix4] = 0
    #e_phases[fix5] -= np.pi
    
    end = time.time()
    elapsed = end - start
    print("Time it took algorithm to run: ",elapsed, "seconds")
    return e_phases, err, k-1

-------------------

## Original Function: Estimated Phases and Error <a id="e-phases-error"></a>

In [52]:
e_phases, err, k = GSA(omg, f_mag, F_mag)

257
Time it took algorithm to run:  0.06380009651184082 seconds


In [27]:
# original function without random initial phase:
e_phases_normal, err_normal, k_normal = GSA(omg_normal, f_mag, F_mag)

257
Time it took algorithm to run:  0.030668020248413086 seconds


## Zero Packed Function: Estimated Phases and Error <a id="e-phases-error-zp"></a>

In [28]:
e_phases_zp, err_zp, k_zp = GSA(omg_zp, f_mag_zp, F_mag_zp)

257
Time it took algorithm to run:  0.04619598388671875 seconds


In [29]:
# zero packed phases without random initial phase:
e_phases_zp_normal, err_zp_normal, k_zp_normal = GSA(omg_zp_normal, f_mag_zp, F_mag_zp)

257
Time it took algorithm to run:  0.04610586166381836 seconds


## Modified Zero Packed Function: Estimated Phases and Error

In [30]:
e_phases_zp1, err_zp1, k_zp1 = GSA(omg_zp1, f_mag_zp1, F_mag_zp1)

257
Time it took algorithm to run:  0.029531002044677734 seconds


In [31]:
e_phases_zp1_edit, err_zp1_edit, k_zp1_edit = GSA(omg_zp1, f_mag_zp1, F_mag_zp1_edit)

257
Time it took algorithm to run:  0.028793811798095703 seconds


In [32]:
# First zp modification without random initial phase:
e_phases_zp1_normal, err_zp1_normal, k_zp1_normal = GSA(omg_zp1_normal, f_mag_zp1, F_mag_zp1)

257
Time it took algorithm to run:  0.030048847198486328 seconds


-------------------

## Original Function: Convergence of Error <a id="conv-error"></a>

In [33]:
# The difference between each error
error_array = []
for i in range(k):
    error_array.append(err[i])

In [34]:
# Error array without normal random initial phase:
error_array_normal = []
for i in range(k_normal):
    error_array_normal.append(err_normal[i])

## Zero Packed Function: Convergence of Error <a id="conv-error-zp"></a>

In [35]:
# The difference between each error
error_array_zp = []
for i in range(k_zp):
    error_array_zp.append(err_zp[i])

In [36]:
# Error array without normal random initial phase:
error_array_zp_normal = []
for i in range(k_zp_normal):
    error_array_zp_normal.append(err_zp_normal[i])

## Modified Zero Packed Function: Convergence of Error

In [37]:
# The difference between each error

# ZP Modification 1:
error_array_zp1 = []
for i in range(k_zp1):
    error_array_zp1.append(err_zp1[i])

In [38]:
# Error array without normal random initial phase:
error_array_zp1_normal = []
for i in range(k_zp1):
    error_array_zp1_normal.append(err_zp1_normal[i])  

-------------------

## Modified ZP Function

In [39]:
# Modification 1
err_zp1_un = unpacking(err_zp1, z)
e_phases_zp1_un = unpacking(e_phases_zp1, z)

In [40]:
err_zp1_edit_un = unpacking(err_zp1_edit, z)

In [41]:
# Modification 1 without random initial phase
err_zp1_un_normal = unpacking(err_zp1_normal, z)
e_phases_zp1_un_normal = unpacking(e_phases_zp1_normal, z)

---------------

# Plots <a id="plots"></a>

In [43]:
# Initialize figure with subplots
fig = make_subplots(rows=2, cols=3, subplot_titles=("Phase Plot", "Estimated Phases", 
                                                    "Convergence of Error", "ZP Phase Plot", 
                                                     "ZP Estimated Phases", "ZP Convergence of Error"))
#Header for Subplots
fig.update_layout(title_text= "The Gerchberg Saxton Algorithm: Original vs. Zero Packing", height=610)
                    
# Plot 1: Phase Plot
fig.add_trace(go.Scatter(x=t, y=phases, mode='markers', name='Original',
                        marker = {'color' : 'blue'}), row=1, col=1)
fig.update_xaxes(title_text="t", row=1, col=1)
fig.update_yaxes(title_text="Phases", row=1, col=1)

# Plot 2: Estimated Phases
#fig.add_trace(go.Scatter(x=t, y=e_phases[0,:], mode='markers', name='Original'),row=1, col=2)
fig.add_trace(go.Scatter(x=t, y=e_phases, mode='markers', name='Original',
                        marker = {'color' : 'blue'}),row=1, col=2)
fig.update_xaxes(title_text="t", row=1, col=2)
fig.update_yaxes(title_text="Phases", row=1, col=2)

# Plot 3: Convergence of Error
x1 = np.arange(2, k) #changed it so it doesnt include the two inital dummy variables
#x1 = np.arange(2, len(err))
#x1 = np.linspace(-1, 1, len(e_phases_zp1_un))
fig.add_trace(go.Scatter(x=x1, y=err, mode='markers', name='Original',
                        marker = {'color' : 'blue'}), row=1, col=3)
fig.update_xaxes(title_text="Iterations", row=1, col=3)
fig.update_yaxes(title_text="Error", row=1, col=3)
                    
# Plot 1: Phase Plot
fig.add_trace(go.Scatter(x=t_zp, y=phases, mode='markers',name='Zero-Packed', 
                        marker = {'color' : 'red'}), row=2, col=1)
fig.update_xaxes(title_text="t", row=2, col=1)
fig.update_yaxes(title_text="Phases", row=2, col=1)

# Plot 2: Estimated Phases
# Note, had issues beforehand and had to change y=e_phases_zp[0,:]
fig.add_trace(go.Scatter(x=t, y=unpacking(e_phases_zp,z), mode='markers', name='Zero-Packed',
                        marker = {'color' : 'red'}),row=2, col=2)
fig.update_xaxes(title_text="t", row=2, col=2)
fig.update_yaxes(title_text="Phases", row=2, col=2)               

# Plot 3: Convergence of Error
x2 = np.arange(2, k_zp)
fig.add_trace(go.Scatter(x=x2, y=unpacking(err_zp,z), mode='markers',name='Zero-Packed',
                         marker = {'color' : 'red'}), row=2, col=3)
fig.update_xaxes(title_text="Iterations", row=2, col=3)
fig.update_yaxes(title_text="Error", row=2, col=3)

In [44]:
# Initialize figure with subplots
fig1 = make_subplots(rows=2, cols=2, subplot_titles=("Original E-Phases", "ZP E-Phases", "ZP Mod1 E-Phases", "ZP Mod2 E-Phases"))
#Header for Subplots
fig1.update_layout(title_text= "Zero-Packing Variations", height=650)
                    
# Plot 1: Unmodified Estimated Phases
fig1.add_trace(go.Scatter(x=t, y=e_phases, mode='markers', name='Original'), row=1, col=1)
fig1.update_xaxes(title_text="t", row=1, col=1)
fig1.update_yaxes(title_text="Estimated Phases", row=1, col=1)

# Plot 2: ZP Estimated Phases
#fig.add_trace(go.Scatter(x=t, y=e_phases[0,:], mode='markers', name='Original'),row=1, col=2)
fig1.add_trace(go.Scatter(x=t, y=unpacking(e_phases_zp,z), mode='markers', name='Zero-Packed'),row=1, col=2)
fig1.update_xaxes(title_text="t", row=1, col=2)
fig1.update_yaxes(title_text="Estimated Phases", row=1, col=2)

# Plot 3: ZP1 Estimated Phases
x = np.linspace(-1,1,len(e_phases_zp1))
x_unpack = np.linspace(-1,1,len(unpacking(e_phases_zp1_edit,z)))
fig1.add_trace(go.Scatter(x=x_unpack, y=unpacking(e_phases_zp1,z), mode='markers', name='Zero-Packed Mod1'), row=2, col=1)
fig1.update_xaxes(title_text="t", row=2, col=1)
fig1.update_yaxes(title_text="Estimated Phases", row=2, col=1)

# Plot 3: ZP1 Estimated Phases
x = np.linspace(-1,1,len(e_phases_zp1_edit))
x_unpack = np.linspace(-1,1,len(unpacking(e_phases_zp1_edit,z)))
#fig1.add_trace(go.Scatter(x=x, y=e_phases_zp1_edit, mode='markers', name='Zero-Packed Mod2'), row=2, col=2)
fig1.add_trace(go.Scatter(x=x_unpack, y=unpacking(e_phases_zp1_edit,z), mode='markers', name='Zero-Packed Mod2'), row=2, col=2)
fig1.update_xaxes(title_text="t", row=2, col=2)
fig1.update_yaxes(title_text="Estimated Phases", row=2, col=2)


* [Back to Top](#top)